# 예외처리 및 고급 예외

# 예외 처리 정리

---

## 1. 오류의 종류
- **구문 오류 (SyntaxError)**  
  - 문법이 잘못되어 코드가 실행되기 전 단계에서 발생  
- **예외 (Exception)**  
  - 실행 중에 발생하는 문제  
  - 예: 파일이 없을 때, 0으로 나눌 때 등  

---

## 2. 기본 예외 처리
```python
try:
    # 문제가 발생할 가능성이 있는 코드
    x = int(input("숫자 입력: "))
    y = 10 / x
except ZeroDivisionError:
    print("0으로 나눌 수 없어요!")
except ValueError:
    print("숫자로 바꿀 수 없어요!")



## 3. `try … except … else` 구문

```python
try:
    result = fetch_data()   # 외부 API 호출 등
except NetworkError:
    print("네트워크 문제 발생!")
else:
    print("데이터 로드 성공:", result)

# else 블록은 try 안의 코드가 예외 없이 모두 성공적으로 실행되었을 때만 실행됨

## 4. `finally` 구문

```python
try:
    conn = open_connection()
    # 작업 수행
except:
    print("문제 발생!")
finally:
    conn.close()   # 항상 실행 → 리소스 정리


## finally 블록

### 예외 발생 여부와 상관없이 항상 실행

### 파일 닫기 · 연결 해제 · 트랜잭션 종료 등 정리(cleanup) 작업에 사용

## 핵심 포인트
- **SyntaxError vs Exception**
- **except 블록 순서**  
  구체적 예외 → 범용 예외 순으로 작성  
- **else**  
  예외 없이 정상 종료되었을 때만 실행  
- **finally**  
  항상 실행, 리소스 정리용  

---

## 생각하기
- **try 없이 except만 쓰면?**  
  - `SyntaxError: 'except' must be preceded by 'try'`  
  - `except`는 “어떤 try 블록에서” 발생한 예외를 잡는 역할을 함  
- **else 블록은 언제 실행되는가?**  
  - `try` 블록 안의 모든 코드가 예외 없이 성공해야만 실행  

---

## 예외 고급
### 왜 예외 고급 처리가 필요한가?
- 단순히 `print`만 하면 어디서 왜 났는지 추적하기 어려움  
- **구체적 예외 객체** 활용 시:  
  1. 오류 타입 분류 (ZeroDivisionError, KeyError 등)  
  2. 상황별 복구 로직 작성 가능  
  3. 디버깅 시 실패한 라인·이유 정확히 파악  


### `except … as e`

```python
except ValueError as e:
    # e는 ValueError 객체
    # e.args 또는 str(e)로 메시지 확인 가능
    print("값 오류:", e)


## e에 실제 예외 객체(클래스 인스턴스)가 할당

## e.args, str(e)를 통해 왜 오류가 났는지 메시지 획득


## 구체적 예외부터 작성 순서 중요

```python
try:
    ...
except Exception:
    # (1) 모든 예외 다 잡음
except ValueError:
    # (2) 절대 도달 못 함!


## **범용 예외(Exception)**를 맨 아래에 두어야 함

## 예외 구분하기 예시

```python
import json

try:
    data = json.loads(input_str)
except json.JSONDecodeError:
    print("잘못된 JSON 형식!")
except KeyError:
    print("필요한 키가 없습니다!")
except Exception:
    print("알 수 없는 오류!")


## JSON 파싱 오류, 키 조회 오류, 그 외 오류를 구체적으로 구분하여 여러분이 처리할 수 있게 작성해보세요!

In [6]:
import json

def process_user_input(input_str):
    try:
        # 1) JSON 파싱
        data = json.loads(input_str)

        # 2) 필수 키 확인
        if "age" not in data:
            raise KeyError("age 키가 없습니다")

        # 3) age 값 처리
        age = data["age"]
        if not isinstance(age, int):
            raise TypeError("age는 정수여야 합니다")

        print(f"사용자 나이: {age}")

    except json.JSONDecodeError:
        # 잘못된 JSON 형식
        print("입력된 문자열이 올바른 JSON이 아닙니다.")
        # → 사용자에게 JSON 포맷 가이드 다시 보여주기 등 복구 로직

    except KeyError as e:
        # 필요한 키가 없을 때
        print(f"키 오류: {e}")
        # → 기본값 설정 or 다시 입력 요청

    except TypeError as e:
        # 타입이 잘못됐을 때
        print(f"타입 오류: {e}")
        # → 정수로 입력하라고 안내

    except Exception as e:
        # 나머지 예외(예상치 못한 오류)
        print("알 수 없는 오류:", e)
        # → 로그에 traceback 남기기 등

    finally:
        print("입력 처리 종료")


In [10]:
# 1) 올바른 JSON & 올바른 age 타입
process_user_input('{"age": 30}')
# 출력:
# 사용자 나이: 30
# 입력 처리 종료
print('\n')
# 2) 잘못된 JSON 포맷
process_user_input('{"age": 30')  
# 출력:
# 입력된 문자열이 올바른 JSON이 아닙니다.
# 입력 처리 종료
print('\n')
# 3) 필수 키(age) 누락
process_user_input('{"name": "Alice"}')
# 출력:
# 키 오류: 'age'
# 입력 처리 종료
print('\n')
# 4) age가 정수가 아닐 때
process_user_input('{"age": "30"}')
# 출력:
# 타입 오류: age는 정수여야 합니다
# 입력 처리 종료
print('\n')
# 5) 기타 예기치 못한 오류 (예: None 전달)
process_user_input(None)
# 출력:
# 알 수 없는 오류: the JSON object must be str, bytes or bytearray, not NoneType
# 입력 처리 종료
print('\n')

사용자 나이: 30
입력 처리 종료


입력된 문자열이 올바른 JSON이 아닙니다.
입력 처리 종료


키 오류: 'age 키가 없습니다'
입력 처리 종료


타입 오류: age는 정수여야 합니다
입력 처리 종료


타입 오류: the JSON object must be str, bytes or bytearray, not NoneType
입력 처리 종료




## 왜 이렇게까지 자세하고 구체적으로 처리?

- **문제 원인 파악**  
  - `json.JSONDecodeError` → 포맷이 틀림  
  - `KeyError` → 필수 데이터 누락  
  - `TypeError` → 타입 불일치  
  - 이 정보를 통해 “사용자에게 어떤 안내를 줄지” 정확히 결정할 수 있습니다.

- **맞춤 복구 로직**  
  - JSON 오류 → “예시 JSON” 템플릿을 재전송  
  - 키 오류 → “age 값을 포함해주세요”  
  - 타입 오류 → “숫자로만 입력해주세요”

- **디버깅 편의성**  
  - 각 `except` 블록에서 `e` 객체(예외 메시지)를 로그로 남기면,  
    “어느 함수, 어떤 입력에서, 어떤 이유로” 실패했는지 나중에 확인하기 쉬워집니다.

---

## 최상위 `except Exception:`

- 모든 예상 외 오류를 한 번 더 잡아보고,  
  “앱이 꺼지지 않도록” 안전망 역할을 하지만,  
- 핵심 로직에서는 가능한 **구체적 예외**를 먼저 처리해야 혼란이 없습니다.

---

## 모든 예외 잡기

- **`except Exception:`**  
  - 파이썬의 거의 모든 표준 예외(`ValueError`, `TypeError` 등)를 잡습니다.  
  - 단, `KeyboardInterrupt`(Ctrl+C)나 `SystemExit` 등은 잡지 않습니다.

- **`except:`** (아무 타입 지정 안 함)  
  - 모든 예외(표준 예외뿐 아니라 시스템 종료, 키보드 인터럽트까지) 전부 잡습니다.

---

## 너무 넓게 잡으면 디버깅 어려워짐

1. **실제 오류 원인 가림**  
   - 버그가 생겨도 무턱대고 `except:`에서 모두 처리해 버리면,  
     “왜 실패했는지” 로그도 안 남기고 넘어갈 수 있어요.

2. **의도치 않은 예외까지 흡수**  
   - 프로그램이 중단되어야 할 심각한 상태까지 계속 실행될 수 있어,  
     뒤에서 더 큰 문제(데이터 손상 등)를 일으킬 수 있습니다.


# raise 구문

## `raise` 구문

- **`raise` 구문의 필요성과 사용 시기**  
- **왜 직접 예외를 발생시키나?**  
  - 함수 내부에서 “이 입력은 잘못됐다!”라고 판단될 때  
  - 호출 코드(함수 쓰는 쪽)에게 “적절한 처리를 맡기기 위해”

- **어떤 상황에 사용하나?**  
  - 함수의 계약(입력 타입·값 조건)을 강제할 때  
  - 하위 모듈에서 에러가 났음을 상위 로직에 알리고 싶을 때


# 파이썬 예외 고급 처리 정리

프로그램 실행 중 예상치 못한 문제가 발생했을 때, 미리 대비하여 안전하게 처리하고 디버깅을 돕기 위해 예외를 배웁니다.

---

## 핵심 포인트

1. **`except Exception as e`**: 예외 객체 `e`로 상세 메시지 활용  
2. **구체적인 예외부터**: `except` 블록은 구체적인 예외 순서대로 작성해야 합니다  
3. **`raise`**: 직접 예외를 던져 호출부에서 처리하게 할 수 있습니다  

---

## 1. 간단한 예외 발생 (`raise`)


In [14]:
def divide(a, b):
    if b == 0:
        raise ZeroDivisionError("0으로 나눌 수 없습니다")
    return a / b

In [16]:
divide(10, 0)

ZeroDivisionError: 0으로 나눌 수 없습니다

## b == 0일 때 직접 ZeroDivisionError를 발생시켜,
## 호출부에서 try/except ZeroDivisionError로 깔끔하게 처리할 수 있습니다.

In [ ]:
# 입력값 검증 예시
def check_input(x):
    if x < 0:
        raise ValueError("음수는 허용되지 않음")
    # x가 0 이상일 때만 이후 로직 실행


## x < 0일 때 ValueError를 던져 호출부에서 처리하도록 합니다.

# 예외 처리 없을 때 vs 있을 때
## 예외 처리 없을 때 (불편)

```python
def run_task():
    try:
        # 1) 설정 파일 읽기
        with open("config.json") as f:
            config = json.load(f)

        # 2) 사용자 입력 받아 나누기
        a = int(input("첫 번째 숫자: "))
        b = int(input("두 번째 숫자: "))
        result = a / b

        # 3) 결과 출력
        print("결과:", result)
    except:
        # 어떤 예외인지, 어디서 났는지 전혀 모름
        print("문제가 발생했습니다.")
    finally:
        print("입력 처리 종료")


## 오류가 어디서 났는지 모름

## config.json이 없을 때인지, int()가 실패했는지, b=0인지 구분 불가

## 개발자가 로그를 남기지 않으면 디버깅이 거의 불가능

# 예외 처리 있을 때 (편리)

In [ ]:
import json
import traceback

def run_task():
    try:
        # 1) 설정 파일 읽기
        with open("config.json") as f:
            config = json.load(f)

        # 2) 사용자 입력 받아 나누기
        a = int(input("첫 번째 숫자: "))
        b = int(input("두 번째 숫자: "))
        result = a / b

        # 3) 결과 출력
        print("결과:", result)

    except FileNotFoundError as e:
        # 설정 파일이 없을 때
        print("설정 파일을 찾을 수 없습니다:", e)
        # 복구 로직: 기본 설정 사용
        config = {"mode": "default"}
        print("기본 설정으로 계속합니다.")

    except ValueError as e:
        # 숫자 변환이 실패했을 때
        print("입력된 값이 숫자가 아닙니다:", e)
        # 복구 로직: 다시 입력받기
        return run_task()

    except ZeroDivisionError as e:
        # 0으로 나눌 때
        print("0으로 나눌 수 없습니다:", e)
        # 복구 로직: b를 1로 설정
        b = 1
        print("b=1로 설정하고 다시 시도합니다.")
        print("결과:", a / b)

    except Exception as e:
        # 나머지 모든 예외
        print("예상치 못한 오류가 발생했습니다.")
        # traceback으로 디버깅 정보 한눈에 보기
        traceback.print_exc()

    finally:
        print("작업이 종료되었습니다.")


## 예외 종류별로 복구 로직을 분리 → 사용자 경험 개선

## traceback.print_exc()로 상세 디버깅 정보 확인 가능

# 호출부에서 직접 예외 발생 처리

In [ ]:
def get_weather(city):
    if not isinstance(city, str):
        # city가 문자열이 아니면 TypeError 예외를 직접 발생시킨다
        raise TypeError("도시 이름은 문자열이어야 합니다")
    # city가 문자열일 때만 API 호출
    return fetch_weather_api(city)

try:
    info = get_weather(123)  # 정수 123을 넣었으므로
except TypeError as e:
    # raise된 TypeError가 잡혀서 이 블록이 실행된다
    print("입력 오류:", e)
